In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

# Resonator length

In [2]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [3]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_2.append(a)
print(resonator_length_2)


[8.103254683870285, 7.874788339841421, 7.658346540209131]


In [5]:
resonator_length_4=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,4)
    a=a.strip('mm')
    a=float(a)
    resonator_length_4.append(a)
print(resonator_length_4)


[4.226627341935142, 4.1123941699207105, 4.004173270104565]


# LOM Analysis

In [11]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [13]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.3',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.3',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [14]:
c1.sim.run(components=['Q0','c0'], open_terminations=[('Q0', 'down'), ('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 02:16PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:16PM [load_ansys_project]: 	Opened Ansys App
INFO 02:16PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 02:16PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project20
INFO 02:16PM [connect_design]: No active design found (or error getting active design).
INFO 02:16PM [connect]: 	 Connected to project "Project20". No design detected
INFO 02:16PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 02:16PM [connect_setup]: 	No design setup detected.
WARNING 02:16PM [connect_setup]: 	Creating Q3D default setup.
INFO 02:16PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:16PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:16PM [analyze]: Analyzing setup Setup
INFO 02:18PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\T

,down_connector_pad_Q0,ground_main_plane,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
down_connector_pad_Q0,46.33500,-30.788,-13.121,-1.5669,-0.11885
ground_main_plane,-30.78800,208.480,-42.364,-41.7740,-32.70600
pad_bot_Q0,-13.12100,-42.364,94.579,-34.4020,-1.81100
pad_top_Q0,-1.56690,-41.774,-34.402,94.9930,-14.32100
quad_coupler_c0,-0.11885,-32.706,-1.811,-14.3210,49.83400


In [15]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.82
c1.setup.freq_bus = [7.26]
c1.run_lom()
c1.lumped_oscillator_all

[2, 3] [4 0]
Predicted Values

Transmon Properties
f_Q 5.630965 [GHz]
EC 297.357631 [MHz]
EJ 14.854145 [GHz]
alpha -341.556118 [MHz]
dispersion 11.790944 [KHz]
Lq 10.995564 [nH]
Cq 65.141184 [fF]
T1 44.132960 [us]

**Coupling Properties**

tCqbus1 -6.229327 [fF]
gbus1_in_MHz -93.199905 [MHz]
χ_bus1 -3.299740 [MHz]
1/T1bus1 3428.035583 [Hz]
T1bus1 46.427448 [us]

tCqbus2 5.800425 [fF]
gbus2_in_MHz 92.388689 [MHz]
χ_bus2 -1.852406 [MHz]
1/T1bus2 178.224715 [Hz]
T1bus2 893.001528 [us]
Bus-Bus Couplings
gbus1_2 6.618682 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.040803,345.425548,14.854145,-402.366605,47.377356,"[-81.85308901537748, 77.19590092224036]","[-5.889870136299853, -2.4536261044561396]",5.889870,81.853089
2,5.986167,338.778911,14.854145,-393.867799,39.802447,"[-91.02136108445997, 88.60101380072176]","[-6.416282153552176, -2.947222844552578]",6.416282,91.021361
3,5.905504,329.101226,14.854145,-381.545387,30.591385,"[-87.42696225111537, 88.44448329596621]","[-4.958277849118817, -2.5739822611020733]",4.958278,87.426962
4,5.795349,316.143871,14.854145,-365.143115,21.098943,"[-89.77860135433184, 87.06302056250064]","[-4.1715341055179, -2.098906810082889]",4.171534,89.778601
5,5.727866,308.352469,14.854145,-355.332898,16.684094,"[-89.9508227212486, 89.34059989832743]","[-3.674985210559726, -1.9960670852421485]",3.674985,89.950823
6,5.697632,304.897576,14.854145,-350.995415,14.990816,"[-92.1682356462014, 90.89300753628477]","[-3.6453459969327238, -1.9755463798494501]",3.645346,92.168236
7,5.662674,300.930488,14.854145,-346.024419,13.226875,"[-92.05357913962754, 91.29546338534473]","[-3.4092675206351584, -1.8936814850690533]",3.409268,92.053579
8,5.649613,299.455891,14.854145,-344.179254,12.61737,"[-93.10875186556567, 92.05636052371291]","[-3.4059607696122023, -1.88924273817706]",3.405961,93.108752
9,5.630965,297.357631,14.854145,-341.556118,11.790944,"[-93.19990480829718, 92.38868874968729]","[-3.299739899289557, -1.852405536514126]",3.299740,93.199905


In [16]:
c1.sim.run(components=['Q1','c1'], open_terminations=[('Q1', 'down'), ('c1', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 02:23PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 02:23PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:23PM [analyze]: Analyzing setup Setup
INFO 02:24PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp969kqljv.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5300000000, Maxwell, 1, False
INFO 02:24PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp5_fav544.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5300000000, Maxwell, 1, False
INFO 02:24PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp5w7n9nis.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5300000000, Maxwell, 2, False
INFO 02:24PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpq4lf_2gg.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 53000000

,down_connector_pad_Q1,ground_main_plane,pad_bot_Q1,pad_top_Q1,quad_coupler_c1
down_connector_pad_Q1,46.32300,-30.752,-13.1500,-1.565,-0.11897
ground_main_plane,-30.75200,208.380,-42.4190,-41.744,-32.56500
pad_bot_Q1,-13.15000,-42.419,94.6290,-34.372,-1.81540
pad_top_Q1,-1.56500,-41.744,-34.3720,94.924,-14.32300
quad_coupler_c1,-0.11897,-32.565,-1.8154,-14.323,49.70200


In [17]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.64
c1.setup.freq_bus = [7.26]
c1.run_lom()
c1.lumped_oscillator_all

[2, 3] [4 0]
Predicted Values

Transmon Properties
f_Q 5.631583 [GHz]
EC 297.427110 [MHz]
EJ 14.854145 [GHz]
alpha -341.642933 [MHz]
dispersion 11.817560 [KHz]
Lq 10.995564 [nH]
Cq 65.125967 [fF]
T1 33.711634 [us]

**Coupling Properties**

tCqbus1 -6.236429 [fF]
gbus1_in_MHz -90.871186 [MHz]
χ_bus1 -4.182942 [MHz]
1/T1bus1 4542.136777 [Hz]
T1bus1 35.039663 [us]

tCqbus2 5.808339 [fF]
gbus2_in_MHz 92.530933 [MHz]
χ_bus2 -1.859768 [MHz]
1/T1bus2 178.931949 [Hz]
T1bus2 889.471915 [us]
Bus-Bus Couplings
gbus1_2 6.457853 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.019728,342.852974,14.854145,-399.073685,44.315138,"[-89.16699160052512, 78.12244649044642]","[-10.07751458730857, -2.424260636173986]",10.077515,89.166992
2,5.970654,336.905157,14.854145,-391.477172,37.859014,"[-88.68393701297852, 87.9054304219207]","[-8.71222027440903, -2.8273634035385458]",8.712220,88.683937
3,5.896334,328.011237,14.854145,-380.161379,29.675824,"[-86.66199169995993, 89.4593000050792]","[-6.869945485105063, -2.595008017306647]",6.869945,86.661992
4,5.824252,319.514907,14.854145,-369.399848,23.291336,"[-87.66344802103252, 90.45681858580551]","[-5.910324784817333, -2.3688186648294147]",5.910325,87.663448
5,5.739241,309.658074,14.854145,-356.974048,17.364418,"[-87.46423867970671, 90.57401877291285]","[-4.857628791017705, -2.0867063829891888]",4.857629,87.464239
6,5.705765,305.824812,14.854145,-352.158765,15.430414,"[-89.59578272536643, 91.83402989584302]","[-4.742685329592318, -2.0410061796179084]",4.742685,89.595783
7,5.665546,301.255349,14.854145,-346.431107,13.364429,"[-89.66664588932976, 91.64566282652947]","[-4.365805227748479, -1.9162283831321638]",4.365805,89.666646
8,5.652505,299.782139,14.854145,-344.587367,12.750143,"[-90.5445660943251, 92.34645195046836]","[-4.333720310284307, -1.9091485416074627]",4.333720,90.544566
9,5.631583,297.42711,14.854145,-341.642933,11.81756,"[-90.87118644454506, 92.53093336662685]","[-4.182941966398044, -1.8597675812904864]",4.182942,90.871186


In [19]:
c1.sim.close()

# Planar

In [4]:
design = designs.DesignPlanar({}, overwrite_enabled=True)
gui = MetalGUI(design)
design.chips.main.size['size_x']='8mm'
design.chips.main.size['size_y']='8mm'

In [5]:
design.chips

{'main': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '8mm',
   'size_y': '8mm',
   'size_z': '-750um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

In [5]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
cp1 = dict(down = dict(loc_W=-1, loc_H=-1, pad_width='120um'))

q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='main',
                                            connection_pads=cp0))]
q1=[TransmonPocket6(design, 'Q1', options=dict(pos_x='2000um',
                                            pos_y='-2000um',
                                            chip='main',
                                            connection_pads=cp1))]
qubits=q0+q1

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))
q1x=float(qubits[1].options.pos_x.strip('um'))
q1y=float(qubits[1].options.pos_y.strip('um'))

c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="main"))
c1 = RightCoupler(design, "c1", options=dict(pos_x=f"{q1x+150}um",
                                              pos_y=f"{q1y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="main")) 

design.rebuild()
gui.rebuild()
gui.autoscale()

In [6]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="main",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="main",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [7]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="main",
                                    down_length='300um',
                                    mirror=True,
                                    open_termination=True))
clt1 = CoupledLineTee(design, "clt1", 
                        options=dict(pos_x="2000um",
                                    pos_y='2000um',
                                    coupling_length='300um', 
                                    chip="main",
                                    down_length='300um',
                                    mirror=False,
                                    open_termination=True))


In [8]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="main", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="main",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='clt1',
                                                            pin='prime_start'))))
fl2=RouteStraight(design, 'fl2',options=Dict(chip="main",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt1',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))

In [9]:
# cpw on main chip
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="main",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [10]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')
rr1=connect('rr1', 'clt1', 'second_end', 'c1', 'in',
            resonator_length_2[0],jogs,jogs, 
            '400um','0um', '100um', '100um')
bus=connect('bus', 'Q0', 'down', 'Q1', 'down',
            resonator_length_2[2],jogs,jogs, 
            '300um','-800um', '100um', '100um')                       

# EPR Analysis

In [12]:
from qiskit_metal.analyses.quantization import EPRanalysis

In [13]:
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [15]:
hfss.start()

INFO 05:02PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:02PM [load_ansys_project]: 	Opened Ansys App
INFO 05:02PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 05:02PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project19
INFO 05:02PM [connect_design]: No active design found (or error getting active design).
INFO 05:02PM [connect]: 	 Connected to project "Project19". No design detected


True

In [16]:
hfss.activate_ansys_design("Planarchip", 'eigenmode')
hfss.render_design(['Q0','Q1','c0','c1','bus','rr0','rr1','fl0','fl1','fl2','clt0','clt1','lpL','lpR'], 
                    [('clt0','prime_end'),('clt1', 'prime_start'),
                    ('c0','in'),('c1','in'),('rr0','end'),('rr1','end'),
                    ('bus','start'),('bus','end')], 
                    box_plus_buffer=True)
                    

05:02PM 23s WARNING [activate_ansys_design]: The design_name=Planarchip was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 05:02PM [connect_design]: 	Opened active design
	Design:    Planarchip [Solution type: Eigenmode]
WARNING 05:02PM [connect_setup]: 	No design setup detected.
WARNING 05:02PM [connect_setup]: 	Creating eigenmode default setup.
INFO 05:02PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [21]:
hfss.close()

NameError: name 'hfss' is not defined